In [36]:
%matplotlib inline

Within-subject SVM classification based on beta weights (per trials) averaged within networks from different grains of MIST parcellation, for CIMAQ memory encoding task (fMRI data).
Mean network betas reflect the engagement of a particular network for each trial.
MIST Parcellations include: 7, 12, 20, 36, 64, 122, 197, 325, 444 networks

Trials (conditions) are classifierd according to:
- task condition (encoding or control task)
- memory performance (hit vs miss, correct vs incorrect source)
- stimulus category (?)

Each model is ran and tested on data from the same subject, and then group statistics (confidence intervals) are computed around accuracy scores from each individual participant.

In [37]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import nilearn
import scipy
import nibabel as nb
import sklearn
import seaborn as sns
import itertools

from numpy import nan as NaN
from matplotlib import pyplot as plt
from nilearn import image, plotting
from nilearn import masking
from nilearn import plotting
from nilearn import datasets
from nilearn.plotting import plot_stat_map, plot_roi, plot_anat, plot_img, show
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, f1_score
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.preprocessing import MinMaxScaler


Step 1: import list of participants, and generate sublists of participants who have enough trials per category for each classification.

Encoding vs Control task conditions (all 94)
Stimulus category (all 94)
Hit versus Miss (42 participants; at least 15 trials per condition)
Correct Source versus Wrong Source (49 participants; at least 15 trials per condition)
Correct Source versus Miss (38 participants; at least 15 trials per condition)
*NOTE: ADD filter to exclude participants with too many scrubbed frames?? *

In [38]:

# Path to directory with participant lists
data_file = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Participants/Splitting/Sub_list.tsv'
sub_data = pd.read_csv(data_file, sep = '\t')

# Exclude participants who failed QC
sub_data = sub_data[sub_data['QC_status']!= 'F']

## ADD filter to exclude participants with too many scrubbed frames?? ##

# Set minimal number of trials needed per subject to include them in analysis
num = 14

# Encoding vs Control, and Stimulus Category classifications
all_subs = sub_data['participant_id']
all_diagnosis = sub_data['cognitive_status']
print(all_subs)
print(len(all_subs))

# Hit versus Miss
hm_data = sub_data[sub_data['hits'] > num]
hm_data = hm_data[hm_data['miss'] > num]
hm_subs = hm_data['participant_id']
hm_diagnosis = hm_data['cognitive_status']
print(hm_subs)
print(len(hm_subs))

# Correct Source versus Wrong Source 
cw_data = sub_data[sub_data['correct_source'] > num]
cw_data = cw_data[cw_data['wrong_source'] > num]
cw_subs = cw_data['participant_id']
cw_diagnosis = cw_data['cognitive_status']
print(cw_subs)
print(len(cw_subs))

# Correct Source versus Miss
cmiss_data = sub_data[sub_data['correct_source'] > num]
cmiss_data = cmiss_data[cmiss_data['miss'] > num]
cmiss_subs = cmiss_data['participant_id']
cmiss_diagnosis = cmiss_data['cognitive_status']
print(cmiss_subs)
print(len(cmiss_subs))


0      108391
1      120839
2      122922
3      127228
4      139593
6      147863
7      150649
8      164965
9      175295
10     178101
11     189005
12     197192
14     199801
15     219637
16     229301
17     247659
18     254402
19     255499
20     258618
21     258912
22     267168
23     270218
24     271596
27     314409
28     326073
29     336665
30     337021
31     350555
32     370092
34     385370
        ...  
70     763590
71     778749
72     783781
73     785217
74     785245
75     804743
77     845675
78     866812
79     878354
80     884343
81     886007
83     893978
85     901551
86     906145
87     914042
88     915022
89     920577
90     932933
91     936730
92     938001
93     955548
94     956049
95     956130
96     968913
97     974246
98     979001
99     983291
100    988602
101    996599
102    998166
Name: participant_id, Length: 94, dtype: int64
94
0      108391
2      122922
4      139593
8      164965
14     199801
17     247659
19     25549

Step 2. Set up paths of directories of interest

Create empty data structures to save and export classification results


In [61]:
# set paths to directories of interest
beta_dir = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Nistats/Betas'
label_dir = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Nistats/Events'
mask_dir = '/Users/mombot/Documents/Simexp/CIMAQ/Data/masks'
output_dir = '/Users/mombot/Documents/Simexp/CIMAQ/Data/Nilearn/Group_results'

# set the parcellation level and load the parcellation map
# 7, 12, 20, 36, 64, 122, 197, 325, 444
numnet = 444

basc_dir = '/Users/mombot/Documents/Simexp/CIMAQ/Data/MIST/Release/Parcellations'
basc = image.load_img(os.path.join(basc_dir, 'MIST_'+str(numnet)+'.nii'))

b_labels = '/Users/mombot/Documents/Simexp/CIMAQ/Data/MIST/Release/Parcel_Information/MIST_'+str(numnet)+'.csv'
basc_labels = pd.read_csv(b_labels, sep=';')


Step 3. ENCODING VERSUS CONTROL TASK CLASSIFICATION

Build and test model for each participant on list, and compile data in a single pandas dataframe

In [44]:
# ENCODING VERSUS CONTROL TASK CLASSIFICATION

# build data structure to store accuracy data and coefficients
enc_ctl_data = pd.DataFrame()
enc_ctl_data.insert(loc = 0, column = 'dccid', value = 'None', allow_duplicates=True)
# enc_ctl_data.insert(loc = 1, column = 'diagnosis', value = 'None', allow_duplicates=True)
for i in range(0, 10):
    enc_ctl_data.insert(loc = enc_ctl_data.shape[1], column = 'CV'+str(i+1)+'_acc', value = NaN, allow_duplicates=True)
enc_ctl_data.insert(loc = enc_ctl_data.shape[1], column = 'TrainSet_MeanCV_acc', value = 'None', allow_duplicates=True)
enc_ctl_data.insert(loc = enc_ctl_data.shape[1], column = 'TestSet_acc', value = 'None', allow_duplicates=True)
netnames = basc_labels['name']
for i in range(0, numnet):
    enc_ctl_data.insert(loc = enc_ctl_data.shape[1], column = netnames[i]+'_coef', value = NaN, allow_duplicates=True)

for sub in all_subs:
    print(sub)
    sub_data = [sub]
    # load subject's beta maps (one per trial)
    betas = image.load_img(img=os.path.join(beta_dir, str(sub), 'TrialContrasts/betas_sub'+str(sub)+'*.nii'),
                           wildcards=True)
    # initialize NiftiLabelMasker object    
    sub_mask = nb.load(os.path.join(mask_dir, 'func_sub'+str(sub)+'_mask_stereonl.nii'))
    sub_label_masker = NiftiLabelsMasker(labels_img=basc, standardize=True, mask_img=sub_mask,
                                         memory = 'nilearn_cache', verbose=0)
                   
    # transform subject's beta maps into vector of network means per trial
    X_enc_ctl = sub_label_masker.fit_transform(betas)

    # load subject's trial labels
    labels_file = os.path.join(label_dir, 'sub-'+str(sub)+'_enco_ctl.tsv')
    enco_ctl_labels = pd.read_csv(labels_file, sep='\t')
    y_enco_ctl = enco_ctl_labels['condition']
    
    # mask data to exclude trials of no interest
    # does not apply here
    
    # Split trials into a training and a test set
    X_train, X_test, y_train, y_test = train_test_split(
        X_enc_ctl, # x
        y_enco_ctl, # y
        test_size = 0.4, # 60%/40% split
        shuffle = True, # shuffle dataset before splitting
        stratify = y_enco_ctl, # keep distribution of conditions consistent betw. train & test sets
        #random_state = 123  # if set number, same shuffle each time, otherwise randomization algo
        ) 
    print('training:', len(X_train), 'testing:', len(X_test))
    print(y_train.value_counts(), y_test.value_counts())
    
    # define the model
    sub_svc = SVC(kernel='linear', class_weight='balanced')
    
    # do cross-validation to evaluate model performance
    # within 10 folds of training set
    # predict
    y_pred = cross_val_predict(sub_svc, X_train, y_train,
                               groups=y_train, cv=10)
    # scores
    cv_acc = cross_val_score(sub_svc, X_train, y_train,
                         groups=y_train, cv=10)
    print(cv_acc)
    
    for i in range(0, len(cv_acc)):
        sub_data.append(cv_acc[i])
        
    # evaluate overall model performance on training data
    overall_acc = accuracy_score(y_pred = y_pred, y_true = y_train)
    overall_cr = classification_report(y_pred = y_pred, y_true = y_train)
    print('Accuracy:',overall_acc)
    print(overall_cr)
    
    sub_data.append(overall_acc)

    # Test model on unseen data from the test set
    sub_svc.fit(X_train, y_train)
    y_pred = sub_svc.predict(X_test) # classify age class using testing data
    acc = sub_svc.score(X_test, y_test) # get accuracy

    cr = classification_report(y_pred=y_pred, y_true=y_test) # get prec., recall & f1
    # print results
    print('accuracy =', acc)
    print(cr)  
    
    sub_data.append(acc)
    
    # get coefficients
    coef_ = sub_svc.coef_[0]
    print(coef_.shape)
    print(coef_)
    
    sub_basc = basc_labels.copy()
    sub_basc.insert(loc=3, column='coef', value=coef_, allow_duplicates=True)
    
    coef = sub_basc['coef']
    for j in range(0, len(coef)):
        sub_data.append(coef[j])

    sub_basc.sort_values(by='coef', axis = 0, ascending = False, inplace=True)
    print(sub_basc.iloc[0:12, 2:4])
    
    enc_ctl_data = enc_ctl_data.append(pd.Series(sub_data, index=enc_ctl_data.columns), ignore_index=True)

enc_ctl_data.to_csv(os.path.join(output_dir, 'SVC_withinSub_enc_ctl_'+str(numnet)+'networks.tsv'),
    sep='\t', header=True, index=False)


108391
training: 70 testing: 47
Enc    47
CTL    23
Name: condition, dtype: int64 Enc    31
CTL    16
Name: condition, dtype: int64
[1.         0.625      0.625      0.71428571 0.71428571 0.85714286
 0.85714286 0.16666667 0.66666667 0.83333333]
Accuracy: 0.7142857142857143
              precision    recall  f1-score   support

         CTL       0.56      0.65      0.60        23
         Enc       0.81      0.74      0.78        47

   micro avg       0.71      0.71      0.71        70
   macro avg       0.68      0.70      0.69        70
weighted avg       0.73      0.71      0.72        70

accuracy = 0.574468085106383
              precision    recall  f1-score   support

         CTL       0.41      0.56      0.47        16
         Enc       0.72      0.58      0.64        31

   micro avg       0.57      0.57      0.57        47
   macro avg       0.56      0.57      0.56        47
weighted avg       0.61      0.57      0.59        47

(7,)
[ 0.11490832  1.00335066 -0.36815126  

training: 70 testing: 47
Enc    47
CTL    23
Name: condition, dtype: int64 Enc    31
CTL    16
Name: condition, dtype: int64
[0.875      0.875      0.5        1.         0.71428571 0.71428571
 0.71428571 0.66666667 0.83333333 0.33333333]
Accuracy: 0.7285714285714285
              precision    recall  f1-score   support

         CTL       0.58      0.61      0.60        23
         Enc       0.80      0.79      0.80        47

   micro avg       0.73      0.73      0.73        70
   macro avg       0.69      0.70      0.70        70
weighted avg       0.73      0.73      0.73        70

accuracy = 0.7021276595744681
              precision    recall  f1-score   support

         CTL       0.56      0.56      0.56        16
         Enc       0.77      0.77      0.77        31

   micro avg       0.70      0.70      0.70        47
   macro avg       0.67      0.67      0.67        47
weighted avg       0.70      0.70      0.70        47

(7,)
[-0.44853032  0.55007606 -0.46621989  0.0929

training: 70 testing: 47
Enc    47
CTL    23
Name: condition, dtype: int64 Enc    31
CTL    16
Name: condition, dtype: int64
[0.375      0.375      0.625      1.         0.57142857 0.57142857
 0.57142857 0.33333333 0.83333333 0.66666667]
Accuracy: 0.5857142857142857
              precision    recall  f1-score   support

         CTL       0.41      0.57      0.47        23
         Enc       0.74      0.60      0.66        47

   micro avg       0.59      0.59      0.59        70
   macro avg       0.57      0.58      0.57        70
weighted avg       0.63      0.59      0.60        70

accuracy = 0.723404255319149
              precision    recall  f1-score   support

         CTL       0.57      0.81      0.67        16
         Enc       0.88      0.68      0.76        31

   micro avg       0.72      0.72      0.72        47
   macro avg       0.72      0.74      0.72        47
weighted avg       0.77      0.72      0.73        47

(7,)
[-0.80049162  0.08144599 -0.37324021  0.62433

training: 70 testing: 47
Enc    47
CTL    23
Name: condition, dtype: int64 Enc    31
CTL    16
Name: condition, dtype: int64
[0.5        0.75       0.75       0.57142857 0.28571429 0.57142857
 0.85714286 0.16666667 0.66666667 0.66666667]
Accuracy: 0.5857142857142857
              precision    recall  f1-score   support

         CTL       0.40      0.52      0.45        23
         Enc       0.72      0.62      0.67        47

   micro avg       0.59      0.59      0.59        70
   macro avg       0.56      0.57      0.56        70
weighted avg       0.62      0.59      0.60        70

accuracy = 0.6170212765957447
              precision    recall  f1-score   support

         CTL       0.44      0.50      0.47        16
         Enc       0.72      0.68      0.70        31

   micro avg       0.62      0.62      0.62        47
   macro avg       0.58      0.59      0.59        47
weighted avg       0.63      0.62      0.62        47

(7,)
[ 0.37319695  0.06938029  0.37288338  0.7182

training: 70 testing: 47
Enc    47
CTL    23
Name: condition, dtype: int64 Enc    31
CTL    16
Name: condition, dtype: int64
[0.375      0.875      0.875      0.57142857 0.28571429 0.71428571
 0.28571429 0.33333333 0.16666667 0.66666667]
Accuracy: 0.5285714285714286
              precision    recall  f1-score   support

         CTL       0.33      0.43      0.38        23
         Enc       0.68      0.57      0.62        47

   micro avg       0.53      0.53      0.53        70
   macro avg       0.50      0.50      0.50        70
weighted avg       0.56      0.53      0.54        70

accuracy = 0.7872340425531915
              precision    recall  f1-score   support

         CTL       0.67      0.75      0.71        16
         Enc       0.86      0.81      0.83        31

   micro avg       0.79      0.79      0.79        47
   macro avg       0.76      0.78      0.77        47
weighted avg       0.80      0.79      0.79        47

(7,)
[ 0.38523837  0.08323424 -0.27847693  0.2936

...

training: 70 testing: 47
Enc    47
CTL    23
Name: condition, dtype: int64 Enc    31
CTL    16
Name: condition, dtype: int64
[0.5        0.625      0.625      0.57142857 0.42857143 0.85714286
 0.42857143 0.66666667 0.83333333 0.66666667]
Accuracy: 0.6142857142857143
              precision    recall  f1-score   support

         CTL       0.45      0.78      0.57        23
         Enc       0.83      0.53      0.65        47

   micro avg       0.61      0.61      0.61        70
   macro avg       0.64      0.66      0.61        70
weighted avg       0.71      0.61      0.62        70

accuracy = 0.6595744680851063
              precision    recall  f1-score   support

         CTL       0.50      0.69      0.58        16
         Enc       0.80      0.65      0.71        31

   micro avg       0.66      0.66      0.66        47
   macro avg       0.65      0.67      0.65        47
weighted avg       0.70      0.66      0.67        47

(7,)
[ 0.35527864 -0.19564537 -0.12203841  0.2956

training: 70 testing: 47
Enc    47
CTL    23
Name: condition, dtype: int64 Enc    31
CTL    16
Name: condition, dtype: int64
[0.25       0.75       0.75       0.71428571 0.85714286 1.
 0.71428571 0.83333333 0.5        0.33333333]
Accuracy: 0.6714285714285714
              precision    recall  f1-score   support

         CTL       0.50      0.70      0.58        23
         Enc       0.82      0.66      0.73        47

   micro avg       0.67      0.67      0.67        70
   macro avg       0.66      0.68      0.66        70
weighted avg       0.71      0.67      0.68        70

accuracy = 0.723404255319149
              precision    recall  f1-score   support

         CTL       0.59      0.62      0.61        16
         Enc       0.80      0.77      0.79        31

   micro avg       0.72      0.72      0.72        47
   macro avg       0.69      0.70      0.70        47
weighted avg       0.73      0.72      0.73        47

(7,)
[0.87305351 0.48613759 0.73668135 0.23044178 0.569648

/anaconda3/envs/may19_py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


training: 70 testing: 47
Enc    47
CTL    23
Name: condition, dtype: int64 Enc    31
CTL    16
Name: condition, dtype: int64
[0.75       0.75       0.875      0.85714286 0.85714286 0.71428571
 1.         0.66666667 0.66666667 0.16666667]
Accuracy: 0.7428571428571429
              precision    recall  f1-score   support

         CTL       0.61      0.61      0.61        23
         Enc       0.81      0.81      0.81        47

   micro avg       0.74      0.74      0.74        70
   macro avg       0.71      0.71      0.71        70
weighted avg       0.74      0.74      0.74        70

accuracy = 0.5531914893617021
              precision    recall  f1-score   support

         CTL       0.35      0.38      0.36        16
         Enc       0.67      0.65      0.66        31

   micro avg       0.55      0.55      0.55        47
   macro avg       0.51      0.51      0.51        47
weighted avg       0.56      0.55      0.56        47

(7,)
[-0.20425692 -0.04165713 -0.24950328  0.3977

training: 70 testing: 47
Enc    47
CTL    23
Name: condition, dtype: int64 Enc    31
CTL    16
Name: condition, dtype: int64
[0.75       0.5        0.625      0.57142857 0.57142857 0.57142857
 0.71428571 0.66666667 0.33333333 0.66666667]
Accuracy: 0.6
              precision    recall  f1-score   support

         CTL       0.42      0.61      0.50        23
         Enc       0.76      0.60      0.67        47

   micro avg       0.60      0.60      0.60        70
   macro avg       0.59      0.60      0.58        70
weighted avg       0.65      0.60      0.61        70

accuracy = 0.6382978723404256
              precision    recall  f1-score   support

         CTL       0.46      0.38      0.41        16
         Enc       0.71      0.77      0.74        31

   micro avg       0.64      0.64      0.64        47
   macro avg       0.58      0.57      0.58        47
weighted avg       0.62      0.64      0.63        47

(7,)
[ 0.48827023  0.91420024 -0.13745868  0.72460491 -0.4429532

In [62]:
# HIT VERSUS MISS TRIAL CLASSIFICATION

# build data structure to store accuracy data and coefficients
hit_miss_data = pd.DataFrame()
hit_miss_data.insert(loc = 0, column = 'dccid', value = 'None', allow_duplicates=True)
# hit_miss_data.insert(loc = 1, column = 'diagnosis', value = 'None', allow_duplicates=True)
for i in range(0, 7):
    hit_miss_data.insert(loc = hit_miss_data.shape[1], column = 'CV'+str(i+1)+'_acc', value = NaN, allow_duplicates=True)
hit_miss_data.insert(loc = hit_miss_data.shape[1], column = 'TrainSet_MeanCV_acc', value = 'None', allow_duplicates=True)
hit_miss_data.insert(loc = hit_miss_data.shape[1], column = 'TestSet_acc', value = 'None', allow_duplicates=True)
netnames = basc_labels['name']
for i in range(0, numnet):
    hit_miss_data.insert(loc = hit_miss_data.shape[1], column = netnames[i]+'_coef', value = NaN, allow_duplicates=True)

for sub in hm_subs:
    print(sub)
    sub_data = [sub]
    # load subject's beta maps (one per trial)
    betas = image.load_img(img=os.path.join(beta_dir, str(sub), 'TrialContrasts/betas_sub'+str(sub)+'*.nii'),
                           wildcards=True)
    # initialize NiftiLabelMasker object    
    sub_mask = nb.load(os.path.join(mask_dir, 'func_sub'+str(sub)+'_mask_stereonl.nii'))
    sub_label_masker = NiftiLabelsMasker(labels_img=basc, standardize=True, mask_img=sub_mask,
                                         memory = 'nilearn_cache', verbose=0)
                   
    # transform subject's beta maps into vector of network means per trial
    X_hit_miss_ctl = sub_label_masker.fit_transform(betas)   
    
    # load subject's trial labels
    labels_file = os.path.join(label_dir, 'sub-'+str(sub)+'_ctl_miss_hit.tsv')
    y_hit_miss_ctl = pd.read_csv(labels_file, sep='\t')
    y_hit_miss_ctl_labels = y_hit_miss_ctl['ctl_miss_hit']
    # mask X and y data to exclude trials of no interest
    hit_miss_mask = y_hit_miss_ctl_labels.isin(['hit', 'missed'])
    y_hit_miss = y_hit_miss_ctl_labels[hit_miss_mask]      
    X_hit_miss  = X_hit_miss_ctl[hit_miss_mask]
    
    # Split trials into a training and a test set
    X_train, X_test, y_train, y_test = train_test_split(
        X_hit_miss, # x
        y_hit_miss, # y
        test_size = 0.4, # 60%/40% split
        shuffle = True, # shuffle dataset before splitting
        stratify = y_hit_miss, # keep distribution of conditions consistent betw. train & test sets
        #random_state = 123  # if set number, same shuffle each time, otherwise randomization algo
        ) 
    print('training:', len(X_train), 'testing:', len(X_test))
    print(y_train.value_counts(), y_test.value_counts())
    
    # define the model
    sub_svc = SVC(kernel='linear', class_weight='balanced')
    
    # do cross-validation to evaluate model performance
    # within 10 folds of training set
    # predict
    y_pred = cross_val_predict(sub_svc, X_train, y_train,
                               groups=y_train, cv=7)
    # scores
    cv_acc = cross_val_score(sub_svc, X_train, y_train,
                         groups=y_train, cv=7)
    print(cv_acc)
    
    for i in range(0, len(cv_acc)):
        sub_data.append(cv_acc[i])
        
    # evaluate overall model performance on training data
    overall_acc = accuracy_score(y_pred = y_pred, y_true = y_train)
    overall_cr = classification_report(y_pred = y_pred, y_true = y_train)
    print('Accuracy:',overall_acc)
    print(overall_cr)
    
    sub_data.append(overall_acc)

    # Test model on unseen data from the test set
    sub_svc.fit(X_train, y_train)
    y_pred = sub_svc.predict(X_test) # classify age class using testing data
    acc = sub_svc.score(X_test, y_test) # get accuracy

    cr = classification_report(y_pred=y_pred, y_true=y_test) # get prec., recall & f1
    # print results
    print('accuracy =', acc)
    print(cr)  
    
    sub_data.append(acc)
    
    # get coefficients
    coef_ = sub_svc.coef_[0]
    print(coef_.shape)
    print(coef_)
    
    sub_basc = basc_labels.copy()
    sub_basc.insert(loc=3, column='coef', value=coef_, allow_duplicates=True)
    
    coef = sub_basc['coef']
    for j in range(0, len(coef)):
        sub_data.append(coef[j])

    sub_basc.sort_values(by='coef', axis = 0, ascending = False, inplace=True)
    print(sub_basc.iloc[0:12, 2:4])
    
    hit_miss_data = hit_miss_data.append(pd.Series(sub_data, index=hit_miss_data.columns), ignore_index=True)

hit_miss_data.to_csv(os.path.join(output_dir, 'SVC_withinSub_hit_miss_'+str(numnet)+'networks.tsv'),
    sep='\t', header=True, index=False)


108391
training: 46 testing: 32
hit       37
missed     9
Name: ctl_miss_hit, dtype: int64 hit       26
missed     6
Name: ctl_miss_hit, dtype: int64
[0.75       0.375      0.83333333 0.83333333 0.83333333 0.33333333
 0.66666667]
Accuracy: 0.6521739130434783
              precision    recall  f1-score   support

         hit       0.77      0.81      0.79        37
      missed       0.00      0.00      0.00         9

   micro avg       0.65      0.65      0.65        46
   macro avg       0.38      0.41      0.39        46
weighted avg       0.62      0.65      0.64        46

accuracy = 0.59375
              precision    recall  f1-score   support

         hit       0.84      0.62      0.71        26
      missed       0.23      0.50      0.32         6

   micro avg       0.59      0.59      0.59        32
   macro avg       0.54      0.56      0.51        32
weighted avg       0.73      0.59      0.64        32

(444,)
[ 2.16744270e-02 -2.81854922e-02  3.93540100e-02  3.70657864e

training: 46 testing: 32
hit       32
missed    14
Name: ctl_miss_hit, dtype: int64 hit       23
missed     9
Name: ctl_miss_hit, dtype: int64
[0.57142857 0.57142857 0.71428571 0.42857143 0.66666667 0.5
 0.33333333]
Accuracy: 0.5434782608695652
              precision    recall  f1-score   support

         hit       0.68      0.66      0.67        32
      missed       0.27      0.29      0.28        14

   micro avg       0.54      0.54      0.54        46
   macro avg       0.47      0.47      0.47        46
weighted avg       0.55      0.54      0.55        46

accuracy = 0.6875
              precision    recall  f1-score   support

         hit       0.74      0.87      0.80        23
      missed       0.40      0.22      0.29         9

   micro avg       0.69      0.69      0.69        32
   macro avg       0.57      0.55      0.54        32
weighted avg       0.64      0.69      0.66        32

(444,)
[-5.57347312e-02  1.72500174e-02  2.02667162e-02 -1.89974442e-02
 -4.6782348

training: 46 testing: 32
missed    24
hit       22
Name: ctl_miss_hit, dtype: int64 missed    17
hit       15
Name: ctl_miss_hit, dtype: int64
[0.375      0.28571429 0.28571429 0.5        0.33333333 0.83333333
 0.5       ]
Accuracy: 0.43478260869565216
              precision    recall  f1-score   support

         hit       0.42      0.45      0.43        22
      missed       0.45      0.42      0.43        24

   micro avg       0.43      0.43      0.43        46
   macro avg       0.44      0.44      0.43        46
weighted avg       0.44      0.43      0.43        46

accuracy = 0.5
              precision    recall  f1-score   support

         hit       0.48      0.67      0.56        15
      missed       0.55      0.35      0.43        17

   micro avg       0.50      0.50      0.50        32
   macro avg       0.51      0.51      0.49        32
weighted avg       0.51      0.50      0.49        32

(444,)
[-3.67399323e-02 -3.99430553e-02 -6.70769143e-02 -4.08005489e-02
 -2.47

training: 46 testing: 32
hit       31
missed    15
Name: ctl_miss_hit, dtype: int64 hit       22
missed    10
Name: ctl_miss_hit, dtype: int64
[0.25       0.57142857 0.28571429 0.5        0.66666667 0.33333333
 0.5       ]
Accuracy: 0.43478260869565216
              precision    recall  f1-score   support

         hit       0.59      0.55      0.57        31
      missed       0.18      0.20      0.19        15

   micro avg       0.43      0.43      0.43        46
   macro avg       0.38      0.37      0.38        46
weighted avg       0.45      0.43      0.44        46

accuracy = 0.71875
              precision    recall  f1-score   support

         hit       0.81      0.77      0.79        22
      missed       0.55      0.60      0.57        10

   micro avg       0.72      0.72      0.72        32
   macro avg       0.68      0.69      0.68        32
weighted avg       0.73      0.72      0.72        32

(444,)
[ 1.02922273e-02 -1.35822415e-02 -2.46424472e-02  3.98396953e-02
  

training: 46 testing: 32
hit       35
missed    11
Name: ctl_miss_hit, dtype: int64 hit       24
missed     8
Name: ctl_miss_hit, dtype: int64
[0.42857143 0.57142857 0.57142857 0.57142857 0.66666667 0.83333333
 0.83333333]
Accuracy: 0.6304347826086957
              precision    recall  f1-score   support

         hit       0.74      0.80      0.77        35
      missed       0.12      0.09      0.11        11

   micro avg       0.63      0.63      0.63        46
   macro avg       0.43      0.45      0.44        46
weighted avg       0.59      0.63      0.61        46

accuracy = 0.625
              precision    recall  f1-score   support

         hit       0.71      0.83      0.77        24
      missed       0.00      0.00      0.00         8

   micro avg       0.62      0.62      0.62        32
   macro avg       0.36      0.42      0.38        32
weighted avg       0.54      0.62      0.58        32

(444,)
[ 4.30701545e-02 -1.25050712e-02 -4.02678311e-02  2.42757873e-02
  4.0

...

training: 46 testing: 32
hit       34
missed    12
Name: ctl_miss_hit, dtype: int64 hit       24
missed     8
Name: ctl_miss_hit, dtype: int64
[0.42857143 0.71428571 0.42857143 0.71428571 0.42857143 0.5
 0.4       ]
Accuracy: 0.5217391304347826
              precision    recall  f1-score   support

         hit       0.68      0.68      0.68        34
      missed       0.08      0.08      0.08        12

   micro avg       0.52      0.52      0.52        46
   macro avg       0.38      0.38      0.38        46
weighted avg       0.52      0.52      0.52        46

accuracy = 0.75
              precision    recall  f1-score   support

         hit       0.86      0.79      0.83        24
      missed       0.50      0.62      0.56         8

   micro avg       0.75      0.75      0.75        32
   macro avg       0.68      0.71      0.69        32
weighted avg       0.77      0.75      0.76        32

(444,)
[ 0.02519062  0.03205005 -0.01221077  0.00498176 -0.01319739 -0.01430318
  0.0

training: 45 testing: 31
hit       27
missed    18
Name: ctl_miss_hit, dtype: int64 hit       19
missed    12
Name: ctl_miss_hit, dtype: int64
[0.71428571 0.42857143 0.71428571 0.57142857 0.83333333 0.33333333
 0.4       ]
Accuracy: 0.5777777777777777
              precision    recall  f1-score   support

         hit       0.67      0.59      0.63        27
      missed       0.48      0.56      0.51        18

   micro avg       0.58      0.58      0.58        45
   macro avg       0.57      0.57      0.57        45
weighted avg       0.59      0.58      0.58        45

accuracy = 0.5483870967741935
              precision    recall  f1-score   support

         hit       0.65      0.58      0.61        19
      missed       0.43      0.50      0.46        12

   micro avg       0.55      0.55      0.55        31
   macro avg       0.54      0.54      0.54        31
weighted avg       0.56      0.55      0.55        31

(444,)
[-1.09896811e-02 -8.15137202e-03  7.10438283e-03 -3.65564

training: 46 testing: 32
hit       27
missed    19
Name: ctl_miss_hit, dtype: int64 hit       18
missed    14
Name: ctl_miss_hit, dtype: int64
[0.57142857 0.71428571 0.42857143 0.71428571 0.42857143 0.5
 0.4       ]
Accuracy: 0.5434782608695652
              precision    recall  f1-score   support

         hit       0.62      0.56      0.59        27
      missed       0.45      0.53      0.49        19

   micro avg       0.54      0.54      0.54        46
   macro avg       0.54      0.54      0.54        46
weighted avg       0.55      0.54      0.55        46

accuracy = 0.625
              precision    recall  f1-score   support

         hit       0.69      0.61      0.65        18
      missed       0.56      0.64      0.60        14

   micro avg       0.62      0.62      0.62        32
   macro avg       0.62      0.63      0.62        32
weighted avg       0.63      0.62      0.63        32

(444,)
[ 0.01982716  0.02300847  0.01936479 -0.02704257 -0.02433474 -0.05743624
 -0.

training: 46 testing: 32
hit       35
missed    11
Name: ctl_miss_hit, dtype: int64 hit       25
missed     7
Name: ctl_miss_hit, dtype: int64
[0.42857143 0.71428571 0.28571429 0.57142857 0.5        0.66666667
 0.66666667]
Accuracy: 0.5434782608695652
              precision    recall  f1-score   support

         hit       0.71      0.69      0.70        35
      missed       0.08      0.09      0.09        11

   micro avg       0.54      0.54      0.54        46
   macro avg       0.39      0.39      0.39        46
weighted avg       0.56      0.54      0.55        46

accuracy = 0.75
              precision    recall  f1-score   support

         hit       0.79      0.92      0.85        25
      missed       0.33      0.14      0.20         7

   micro avg       0.75      0.75      0.75        32
   macro avg       0.56      0.53      0.53        32
weighted avg       0.69      0.75      0.71        32

(444,)
[-1.53356527e-02  6.56610368e-02 -3.33885263e-03  1.97764042e-02
 -2.50

training: 44 testing: 30
hit       27
missed    17
Name: ctl_miss_hit, dtype: int64 hit       19
missed    11
Name: ctl_miss_hit, dtype: int64
[0.71428571 0.57142857 0.28571429 0.33333333 0.66666667 0.33333333
 0.4       ]
Accuracy: 0.4772727272727273
              precision    recall  f1-score   support

         hit       0.57      0.59      0.58        27
      missed       0.31      0.29      0.30        17

   micro avg       0.48      0.48      0.48        44
   macro avg       0.44      0.44      0.44        44
weighted avg       0.47      0.48      0.47        44

accuracy = 0.6
              precision    recall  f1-score   support

         hit       0.64      0.84      0.73        19
      missed       0.40      0.18      0.25        11

   micro avg       0.60      0.60      0.60        30
   macro avg       0.52      0.51      0.49        30
weighted avg       0.55      0.60      0.55        30

(444,)
[ 3.68994661e-02 -3.54433557e-03 -2.92435363e-02  4.81398808e-02
  1.635

In [63]:
# CORRECT SOURCE VERSUS WRONG SOURCE TRIAL CLASSIFICATION

# build data structure to store accuracy data and coefficients
cs_ws_data = pd.DataFrame()
cs_ws_data.insert(loc = 0, column = 'dccid', value = 'None', allow_duplicates=True)
# cs_ws_data.insert(loc = 1, column = 'diagnosis', value = 'None', allow_duplicates=True)
for i in range(0, 7):
    cs_ws_data.insert(loc = cs_ws_data.shape[1], column = 'CV'+str(i+1)+'_acc', value = NaN, allow_duplicates=True)
cs_ws_data.insert(loc = cs_ws_data.shape[1], column = 'TrainSet_MeanCV_acc', value = 'None', allow_duplicates=True)
cs_ws_data.insert(loc = cs_ws_data.shape[1], column = 'TestSet_acc', value = 'None', allow_duplicates=True)
netnames = basc_labels['name']
for i in range(0, numnet):
    cs_ws_data.insert(loc = cs_ws_data.shape[1], column = netnames[i]+'_coef', value = NaN, allow_duplicates=True)

for sub in cw_subs:
    print(sub)
    sub_data = [sub]
    # load subject's beta maps (one per trial)
    betas = image.load_img(img=os.path.join(beta_dir, str(sub), 'TrialContrasts/betas_sub'+str(sub)+'*.nii'),
                           wildcards=True)
    # initialize NiftiLabelMasker object    
    sub_mask = nb.load(os.path.join(mask_dir, 'func_sub'+str(sub)+'_mask_stereonl.nii'))
    sub_label_masker = NiftiLabelsMasker(labels_img=basc, standardize=True, mask_img=sub_mask,
                                         memory = 'nilearn_cache', verbose=0)
                   
    # transform subject's beta maps into vector of network means per trial
    X_cs_ws_miss_ctl = sub_label_masker.fit_transform(betas)   
    
    # load subject's trial labels
    labels_file = os.path.join(label_dir, 'sub-'+str(sub)+'_ctl_miss_ws_cs.tsv')
    y_cs_ws_miss_ctl = pd.read_csv(labels_file, sep='\t')
    y_cs_ws_miss_ctl_labels = y_cs_ws_miss_ctl['ctl_miss_ws_cs']
    # mask X and y data to exclude trials of no interest
    cs_ws_mask = y_cs_ws_miss_ctl_labels.isin(['correctsource', 'wrongsource'])
    y_cs_ws = y_cs_ws_miss_ctl_labels[cs_ws_mask]      
    X_cs_ws  = X_cs_ws_miss_ctl[cs_ws_mask]
    
    # Split trials into a training and a test set
    X_train, X_test, y_train, y_test = train_test_split(
        X_cs_ws, # x
        y_cs_ws, # y
        test_size = 0.4, # 60%/40% split
        shuffle = True, # shuffle dataset before splitting
        stratify = y_cs_ws, # keep distribution of conditions consistent betw. train & test sets
        #random_state = 123  # if set number, same shuffle each time, otherwise randomization algo
        ) 
    print('training:', len(X_train), 'testing:', len(X_test))
    print(y_train.value_counts(), y_test.value_counts())
    
    # define the model
    sub_svc = SVC(kernel='linear', class_weight='balanced')
    
    # do cross-validation to evaluate model performance
    # within 10 folds of training set
    # predict
    y_pred = cross_val_predict(sub_svc, X_train, y_train,
                               groups=y_train, cv=7)
    # scores
    cv_acc = cross_val_score(sub_svc, X_train, y_train,
                         groups=y_train, cv=7)
    print(cv_acc)
    
    for i in range(0, len(cv_acc)):
        sub_data.append(cv_acc[i])
        
    # evaluate overall model performance on training data
    overall_acc = accuracy_score(y_pred = y_pred, y_true = y_train)
    overall_cr = classification_report(y_pred = y_pred, y_true = y_train)
    print('Accuracy:',overall_acc)
    print(overall_cr)
    
    sub_data.append(overall_acc)

    # Test model on unseen data from the test set
    sub_svc.fit(X_train, y_train)
    y_pred = sub_svc.predict(X_test) # classify age class using testing data
    acc = sub_svc.score(X_test, y_test) # get accuracy

    cr = classification_report(y_pred=y_pred, y_true=y_test) # get prec., recall & f1
    # print results
    print('accuracy =', acc)
    print(cr)  
    
    sub_data.append(acc)
    
    # get coefficients
    coef_ = sub_svc.coef_[0]
    print(coef_.shape)
    print(coef_)
    
    sub_basc = basc_labels.copy()
    sub_basc.insert(loc=3, column='coef', value=coef_, allow_duplicates=True)
    
    coef = sub_basc['coef']
    for j in range(0, len(coef)):
        sub_data.append(coef[j])

    sub_basc.sort_values(by='coef', axis = 0, ascending = False, inplace=True)
    print(sub_basc.iloc[0:12, 2:4])
    
    cs_ws_data = cs_ws_data.append(pd.Series(sub_data, index=cs_ws_data.columns), ignore_index=True)
    
cs_ws_data.to_csv(os.path.join(output_dir, 'SVC_withinSub_cs_ws_'+str(numnet)+'networks.tsv'),
    sep='\t', header=True, index=False)


108391
training: 37 testing: 26
correctsource    25
wrongsource      12
Name: ctl_miss_ws_cs, dtype: int64 correctsource    18
wrongsource       8
Name: ctl_miss_ws_cs, dtype: int64
[0.66666667 0.66666667 0.33333333 0.33333333 0.2        0.5
 0.5       ]
Accuracy: 0.4594594594594595
               precision    recall  f1-score   support

correctsource       0.61      0.56      0.58        25
  wrongsource       0.21      0.25      0.23        12

    micro avg       0.46      0.46      0.46        37
    macro avg       0.41      0.41      0.41        37
 weighted avg       0.48      0.46      0.47        37

accuracy = 0.5384615384615384
               precision    recall  f1-score   support

correctsource       0.69      0.61      0.65        18
  wrongsource       0.30      0.38      0.33         8

    micro avg       0.54      0.54      0.54        26
    macro avg       0.49      0.49      0.49        26
 weighted avg       0.57      0.54      0.55        26

(444,)
[-1.48645705e

training: 33 testing: 22
wrongsource      18
correctsource    15
Name: ctl_miss_ws_cs, dtype: int64 wrongsource      12
correctsource    10
Name: ctl_miss_ws_cs, dtype: int64
[0.83333333 0.4        0.8        0.6        0.5        0.25
 0.5       ]
Accuracy: 0.5757575757575758
               precision    recall  f1-score   support

correctsource       0.53      0.67      0.59        15
  wrongsource       0.64      0.50      0.56        18

    micro avg       0.58      0.58      0.58        33
    macro avg       0.58      0.58      0.58        33
 weighted avg       0.59      0.58      0.57        33

accuracy = 0.36363636363636365
               precision    recall  f1-score   support

correctsource       0.25      0.20      0.22        10
  wrongsource       0.43      0.50      0.46        12

    micro avg       0.36      0.36      0.36        22
    macro avg       0.34      0.35      0.34        22
 weighted avg       0.35      0.36      0.35        22

(444,)
[ 0.013788   -0.00

training: 38 testing: 26
wrongsource      26
correctsource    12
Name: ctl_miss_ws_cs, dtype: int64 wrongsource      17
correctsource     9
Name: ctl_miss_ws_cs, dtype: int64
[0.5        0.66666667 0.16666667 0.33333333 0.66666667 0.75
 0.75      ]
Accuracy: 0.5263157894736842
               precision    recall  f1-score   support

correctsource       0.25      0.25      0.25        12
  wrongsource       0.65      0.65      0.65        26

    micro avg       0.53      0.53      0.53        38
    macro avg       0.45      0.45      0.45        38
 weighted avg       0.53      0.53      0.53        38

accuracy = 0.6923076923076923
               precision    recall  f1-score   support

correctsource       0.54      0.78      0.64         9
  wrongsource       0.85      0.65      0.73        17

    micro avg       0.69      0.69      0.69        26
    macro avg       0.69      0.71      0.68        26
 weighted avg       0.74      0.69      0.70        26

(444,)
[-4.14538142e-02  9

training: 42 testing: 29
correctsource    30
wrongsource      12
Name: ctl_miss_ws_cs, dtype: int64 correctsource    21
wrongsource       8
Name: ctl_miss_ws_cs, dtype: int64
[0.71428571 0.71428571 0.5        0.83333333 0.83333333 1.
 1.        ]
Accuracy: 0.7857142857142857
               precision    recall  f1-score   support

correctsource       0.86      0.83      0.85        30
  wrongsource       0.62      0.67      0.64        12

    micro avg       0.79      0.79      0.79        42
    macro avg       0.74      0.75      0.74        42
 weighted avg       0.79      0.79      0.79        42

accuracy = 0.5517241379310345
               precision    recall  f1-score   support

correctsource       0.68      0.71      0.70        21
  wrongsource       0.14      0.12      0.13         8

    micro avg       0.55      0.55      0.55        29
    macro avg       0.41      0.42      0.42        29
 weighted avg       0.53      0.55      0.54        29

(444,)
[-1.91974155e-02  6.2

training: 43 testing: 29
correctsource    23
wrongsource      20
Name: ctl_miss_ws_cs, dtype: int64 correctsource    16
wrongsource      13
Name: ctl_miss_ws_cs, dtype: int64
[0.71428571 0.57142857 0.33333333 0.5        0.16666667 1.
 0.6       ]
Accuracy: 0.5581395348837209
               precision    recall  f1-score   support

correctsource       0.58      0.61      0.60        23
  wrongsource       0.53      0.50      0.51        20

    micro avg       0.56      0.56      0.56        43
    macro avg       0.55      0.55      0.55        43
 weighted avg       0.56      0.56      0.56        43

accuracy = 0.5862068965517241
               precision    recall  f1-score   support

correctsource       0.62      0.62      0.62        16
  wrongsource       0.54      0.54      0.54        13

    micro avg       0.59      0.59      0.59        29
    macro avg       0.58      0.58      0.58        29
 weighted avg       0.59      0.59      0.59        29

(444,)
[-2.55523175e-02  1.2

...

training: 40 testing: 27
correctsource    23
wrongsource      17
Name: ctl_miss_ws_cs, dtype: int64 correctsource    15
wrongsource      12
Name: ctl_miss_ws_cs, dtype: int64
[0.57142857 0.42857143 0.33333333 0.6        0.8        0.6
 0.8       ]
Accuracy: 0.575
               precision    recall  f1-score   support

correctsource       0.64      0.61      0.62        23
  wrongsource       0.50      0.53      0.51        17

    micro avg       0.57      0.57      0.57        40
    macro avg       0.57      0.57      0.57        40
 weighted avg       0.58      0.57      0.58        40

accuracy = 0.4444444444444444
               precision    recall  f1-score   support

correctsource       0.50      0.67      0.57        15
  wrongsource       0.29      0.17      0.21        12

    micro avg       0.44      0.44      0.44        27
    macro avg       0.39      0.42      0.39        27
 weighted avg       0.40      0.44      0.41        27

(444,)
[ 9.87416313e-03 -2.99337783e-02 

training: 27 testing: 19
correctsource    18
wrongsource       9
Name: ctl_miss_ws_cs, dtype: int64 correctsource    12
wrongsource       7
Name: ctl_miss_ws_cs, dtype: int64
[0.8        0.6        0.5        0.75       0.66666667 0.33333333
 0.        ]
Accuracy: 0.5555555555555556
               precision    recall  f1-score   support

correctsource       0.67      0.67      0.67        18
  wrongsource       0.33      0.33      0.33         9

    micro avg       0.56      0.56      0.56        27
    macro avg       0.50      0.50      0.50        27
 weighted avg       0.56      0.56      0.56        27

accuracy = 0.5789473684210527
               precision    recall  f1-score   support

correctsource       0.67      0.67      0.67        12
  wrongsource       0.43      0.43      0.43         7

    micro avg       0.58      0.58      0.58        19
    macro avg       0.55      0.55      0.55        19
 weighted avg       0.58      0.58      0.58        19

(444,)
[-1.26789509e

training: 43 testing: 30
correctsource    29
wrongsource      14
Name: ctl_miss_ws_cs, dtype: int64 correctsource    20
wrongsource      10
Name: ctl_miss_ws_cs, dtype: int64
[0.57142857 0.5        0.83333333 0.5        0.5        0.5
 0.5       ]
Accuracy: 0.5581395348837209
               precision    recall  f1-score   support

correctsource       0.68      0.66      0.67        29
  wrongsource       0.33      0.36      0.34        14

    micro avg       0.56      0.56      0.56        43
    macro avg       0.51      0.51      0.51        43
 weighted avg       0.57      0.56      0.56        43

accuracy = 0.6
               precision    recall  f1-score   support

correctsource       0.67      0.80      0.73        20
  wrongsource       0.33      0.20      0.25        10

    micro avg       0.60      0.60      0.60        30
    macro avg       0.50      0.50      0.49        30
 weighted avg       0.56      0.60      0.57        30

(444,)
[ 1.82622661e-02 -1.01098910e-03 -2

training: 46 testing: 31
correctsource    35
wrongsource      11
Name: ctl_miss_ws_cs, dtype: int64 correctsource    23
wrongsource       8
Name: ctl_miss_ws_cs, dtype: int64
[0.57142857 0.57142857 0.71428571 0.71428571 0.66666667 1.
 0.5       ]
Accuracy: 0.6739130434782609
               precision    recall  f1-score   support

correctsource       0.75      0.86      0.80        35
  wrongsource       0.17      0.09      0.12        11

    micro avg       0.67      0.67      0.67        46
    macro avg       0.46      0.47      0.46        46
 weighted avg       0.61      0.67      0.64        46

accuracy = 0.7741935483870968
               precision    recall  f1-score   support

correctsource       0.81      0.91      0.86        23
  wrongsource       0.60      0.38      0.46         8

    micro avg       0.77      0.77      0.77        31
    macro avg       0.70      0.64      0.66        31
 weighted avg       0.75      0.77      0.76        31

(444,)
[-0.04482584  0.01395

training: 40 testing: 27
correctsource    25
wrongsource      15
Name: ctl_miss_ws_cs, dtype: int64 correctsource    17
wrongsource      10
Name: ctl_miss_ws_cs, dtype: int64
[0.57142857 0.5        0.66666667 0.16666667 0.8        0.6
 0.6       ]
Accuracy: 0.55
               precision    recall  f1-score   support

correctsource       0.63      0.68      0.65        25
  wrongsource       0.38      0.33      0.36        15

    micro avg       0.55      0.55      0.55        40
    macro avg       0.51      0.51      0.51        40
 weighted avg       0.54      0.55      0.54        40

accuracy = 0.48148148148148145
               precision    recall  f1-score   support

correctsource       0.59      0.59      0.59        17
  wrongsource       0.30      0.30      0.30        10

    micro avg       0.48      0.48      0.48        27
    macro avg       0.44      0.44      0.44        27
 weighted avg       0.48      0.48      0.48        27

(444,)
[ 1.94760587e-02 -3.85237519e-02 

In [64]:
# CORRECT SOURCE VERSUS MISSED TRIAL CLASSIFICATION

# build data structure to store accuracy data and coefficients
cs_miss_data = pd.DataFrame()
cs_miss_data.insert(loc = 0, column = 'dccid', value = 'None', allow_duplicates=True)
# cs_miss_data.insert(loc = 1, column = 'diagnosis', value = 'None', allow_duplicates=True)
for i in range(0, 7):
    cs_miss_data.insert(loc = cs_miss_data.shape[1], column = 'CV'+str(i+1)+'_acc', value = NaN, allow_duplicates=True)
cs_miss_data.insert(loc = cs_miss_data.shape[1], column = 'TrainSet_MeanCV_acc', value = 'None', allow_duplicates=True)
cs_miss_data.insert(loc = cs_miss_data.shape[1], column = 'TestSet_acc', value = 'None', allow_duplicates=True)
netnames = basc_labels['name']
for i in range(0, numnet):
    cs_miss_data.insert(loc = cs_miss_data.shape[1], column = netnames[i]+'_coef', value = NaN, allow_duplicates=True)

for sub in cmiss_subs:
    print(sub)
    sub_data = [sub]
    # load subject's beta maps (one per trial)
    betas = image.load_img(img=os.path.join(beta_dir, str(sub), 'TrialContrasts/betas_sub'+str(sub)+'*.nii'),
                           wildcards=True)
    # initialize NiftiLabelMasker object    
    sub_mask = nb.load(os.path.join(mask_dir, 'func_sub'+str(sub)+'_mask_stereonl.nii'))
    sub_label_masker = NiftiLabelsMasker(labels_img=basc, standardize=True, mask_img=sub_mask,
                                         memory = 'nilearn_cache', verbose=0)
                   
    # transform subject's beta maps into vector of network means per trial
    X_cs_ws_miss_ctl = sub_label_masker.fit_transform(betas)   
    
    # load subject's trial labels
    labels_file = os.path.join(label_dir, 'sub-'+str(sub)+'_ctl_miss_ws_cs.tsv')
    y_cs_ws_miss_ctl = pd.read_csv(labels_file, sep='\t')
    y_cs_ws_miss_ctl_labels = y_cs_ws_miss_ctl['ctl_miss_ws_cs']
    # mask X and y data to exclude trials of no interest
    cs_miss_mask = y_cs_ws_miss_ctl_labels.isin(['correctsource', 'missed'])
    y_cs_miss = y_cs_ws_miss_ctl_labels[cs_miss_mask]      
    X_cs_miss  = X_cs_ws_miss_ctl[cs_miss_mask]
    
    # Split trials into a training and a test set
    X_train, X_test, y_train, y_test = train_test_split(
        X_cs_miss, # x
        y_cs_miss, # y
        test_size = 0.4, # 60%/40% split
        shuffle = True, # shuffle dataset before splitting
        stratify = y_cs_miss, # keep distribution of conditions consistent betw. train & test sets
        #random_state = 123  # if set number, same shuffle each time, otherwise randomization algo
        ) 
    print('training:', len(X_train), 'testing:', len(X_test))
    print(y_train.value_counts(), y_test.value_counts())
    
    # define the model
    sub_svc = SVC(kernel='linear', class_weight='balanced')
    
    # do cross-validation to evaluate model performance
    # within 10 folds of training set
    # predict
    y_pred = cross_val_predict(sub_svc, X_train, y_train,
                               groups=y_train, cv=7)
    # scores
    cv_acc = cross_val_score(sub_svc, X_train, y_train,
                         groups=y_train, cv=7)
    print(cv_acc)
    
    for i in range(0, len(cv_acc)):
        sub_data.append(cv_acc[i])
        
    # evaluate overall model performance on training data
    overall_acc = accuracy_score(y_pred = y_pred, y_true = y_train)
    overall_cr = classification_report(y_pred = y_pred, y_true = y_train)
    print('Accuracy:',overall_acc)
    print(overall_cr)
    
    sub_data.append(overall_acc)

    # Test model on unseen data from the test set
    sub_svc.fit(X_train, y_train)
    y_pred = sub_svc.predict(X_test) # classify age class using testing data
    acc = sub_svc.score(X_test, y_test) # get accuracy

    cr = classification_report(y_pred=y_pred, y_true=y_test) # get prec., recall & f1
    # print results
    print('accuracy =', acc)
    print(cr)  
    
    sub_data.append(acc)
    
    # get coefficients
    coef_ = sub_svc.coef_[0]
    print(coef_.shape)
    print(coef_)
    
    sub_basc = basc_labels.copy()
    sub_basc.insert(loc=3, column='coef', value=coef_, allow_duplicates=True)
    
    coef = sub_basc['coef']
    for j in range(0, len(coef)):
        sub_data.append(coef[j])

    sub_basc.sort_values(by='coef', axis = 0, ascending = False, inplace=True)
    print(sub_basc.iloc[0:12, 2:4])
    
    cs_miss_data = cs_miss_data.append(pd.Series(sub_data, index=cs_ws_data.columns), ignore_index=True)
    
cs_miss_data.to_csv(os.path.join(output_dir, 'SVC_withinSub_cs_miss_'+str(numnet)+'networks.tsv'),
    sep='\t', header=True, index=False)


108391
training: 34 testing: 24
correctsource    25
missed            9
Name: ctl_miss_ws_cs, dtype: int64 correctsource    18
missed            6
Name: ctl_miss_ws_cs, dtype: int64
[0.5  0.5  0.8  0.2  0.75 0.5  0.75]
Accuracy: 0.5588235294117647
               precision    recall  f1-score   support

correctsource       0.69      0.72      0.71        25
       missed       0.12      0.11      0.12         9

    micro avg       0.56      0.56      0.56        34
    macro avg       0.41      0.42      0.41        34
 weighted avg       0.54      0.56      0.55        34

accuracy = 0.625
               precision    recall  f1-score   support

correctsource       0.74      0.78      0.76        18
       missed       0.20      0.17      0.18         6

    micro avg       0.62      0.62      0.62        24
    macro avg       0.47      0.47      0.47        24
 weighted avg       0.60      0.62      0.61        24

(444,)
[ 2.31009846e-02  4.86474945e-03  4.10324176e-02  5.00142178e-

training: 28 testing: 20
correctsource    15
missed           13
Name: ctl_miss_ws_cs, dtype: int64 missed           10
correctsource    10
Name: ctl_miss_ws_cs, dtype: int64
[0.2        0.5        0.5        0.75       0.75       0.75
 0.66666667]
Accuracy: 0.5714285714285714
               precision    recall  f1-score   support

correctsource       0.60      0.60      0.60        15
       missed       0.54      0.54      0.54        13

    micro avg       0.57      0.57      0.57        28
    macro avg       0.57      0.57      0.57        28
 weighted avg       0.57      0.57      0.57        28

accuracy = 0.65
               precision    recall  f1-score   support

correctsource       0.67      0.60      0.63        10
       missed       0.64      0.70      0.67        10

    micro avg       0.65      0.65      0.65        20
    macro avg       0.65      0.65      0.65        20
 weighted avg       0.65      0.65      0.65        20

(444,)
[ 7.09160935e-03 -1.62784125e-02 

training: 33 testing: 23
missed           24
correctsource     9
Name: ctl_miss_ws_cs, dtype: int64 missed           17
correctsource     6
Name: ctl_miss_ws_cs, dtype: int64
[0.83333333 0.5        0.6        0.5        0.5        0.25
 0.75      ]
Accuracy: 0.5757575757575758
               precision    recall  f1-score   support

correctsource       0.22      0.22      0.22         9
       missed       0.71      0.71      0.71        24

    micro avg       0.58      0.58      0.58        33
    macro avg       0.47      0.47      0.47        33
 weighted avg       0.58      0.58      0.58        33

accuracy = 0.5652173913043478
               precision    recall  f1-score   support

correctsource       0.25      0.33      0.29         6
       missed       0.73      0.65      0.69        17

    micro avg       0.57      0.57      0.57        23
    macro avg       0.49      0.49      0.49        23
 weighted avg       0.61      0.57      0.58        23

(444,)
[ 6.37861376e-03  3

training: 39 testing: 27
correctsource    24
missed           15
Name: ctl_miss_ws_cs, dtype: int64 correctsource    17
missed           10
Name: ctl_miss_ws_cs, dtype: int64
[0.14285714 0.33333333 0.83333333 0.8        0.8        0.4
 0.4       ]
Accuracy: 0.5128205128205128
               precision    recall  f1-score   support

correctsource       0.60      0.62      0.61        24
       missed       0.36      0.33      0.34        15

    micro avg       0.51      0.51      0.51        39
    macro avg       0.48      0.48      0.48        39
 weighted avg       0.51      0.51      0.51        39

accuracy = 0.6296296296296297
               precision    recall  f1-score   support

correctsource       0.65      0.88      0.75        17
       missed       0.50      0.20      0.29        10

    micro avg       0.63      0.63      0.63        27
    macro avg       0.58      0.54      0.52        27
 weighted avg       0.60      0.63      0.58        27

(444,)
[-3.72392379e-02 -8.

training: 20 testing: 14
missed           11
correctsource     9
Name: ctl_miss_ws_cs, dtype: int64 missed           8
correctsource    6
Name: ctl_miss_ws_cs, dtype: int64
[0.         0.5        0.66666667 0.33333333 0.5        0.5
 0.5       ]
Accuracy: 0.4
               precision    recall  f1-score   support

correctsource       0.33      0.33      0.33         9
       missed       0.45      0.45      0.45        11

    micro avg       0.40      0.40      0.40        20
    macro avg       0.39      0.39      0.39        20
 weighted avg       0.40      0.40      0.40        20

accuracy = 0.5
               precision    recall  f1-score   support

correctsource       0.40      0.33      0.36         6
       missed       0.56      0.62      0.59         8

    micro avg       0.50      0.50      0.50        14
    macro avg       0.48      0.48      0.48        14
 weighted avg       0.49      0.50      0.49        14

(444,)
[ 1.11729369e-02 -7.23722580e-03  7.03709048e-03  1.

...

training: 39 testing: 26
correctsource    27
missed           12
Name: ctl_miss_ws_cs, dtype: int64 correctsource    18
missed            8
Name: ctl_miss_ws_cs, dtype: int64
[0.83333333 0.66666667 0.83333333 0.33333333 0.83333333 0.8
 1.        ]
Accuracy: 0.7435897435897436
               precision    recall  f1-score   support

correctsource       0.79      0.85      0.82        27
       missed       0.60      0.50      0.55        12

    micro avg       0.74      0.74      0.74        39
    macro avg       0.70      0.68      0.68        39
 weighted avg       0.73      0.74      0.74        39

accuracy = 0.6153846153846154
               precision    recall  f1-score   support

correctsource       0.68      0.83      0.75        18
       missed       0.25      0.12      0.17         8

    micro avg       0.62      0.62      0.62        26
    macro avg       0.47      0.48      0.46        26
 weighted avg       0.55      0.62      0.57        26

(444,)
[-1.22664458e-03  4.

training: 36 testing: 24
missed           18
correctsource    18
Name: ctl_miss_ws_cs, dtype: int64 missed           12
correctsource    12
Name: ctl_miss_ws_cs, dtype: int64
[0.5        0.33333333 0.83333333 0.33333333 0.5        0.5
 0.75      ]
Accuracy: 0.5277777777777778
               precision    recall  f1-score   support

correctsource       0.53      0.44      0.48        18
       missed       0.52      0.61      0.56        18

    micro avg       0.53      0.53      0.53        36
    macro avg       0.53      0.53      0.52        36
 weighted avg       0.53      0.53      0.52        36

accuracy = 0.625
               precision    recall  f1-score   support

correctsource       0.67      0.50      0.57        12
       missed       0.60      0.75      0.67        12

    micro avg       0.62      0.62      0.62        24
    macro avg       0.63      0.62      0.62        24
 weighted avg       0.63      0.62      0.62        24

(444,)
[ 3.44729461e-02 -6.40316829e-02 

training: 38 testing: 26
missed           20
correctsource    18
Name: ctl_miss_ws_cs, dtype: int64 missed           13
correctsource    13
Name: ctl_miss_ws_cs, dtype: int64
[0.66666667 0.5        0.5        1.         0.6        0.4
 0.25      ]
Accuracy: 0.5789473684210527
               precision    recall  f1-score   support

correctsource       0.57      0.44      0.50        18
       missed       0.58      0.70      0.64        20

    micro avg       0.58      0.58      0.58        38
    macro avg       0.58      0.57      0.57        38
 weighted avg       0.58      0.58      0.57        38

accuracy = 0.4230769230769231
               precision    recall  f1-score   support

correctsource       0.40      0.31      0.35        13
       missed       0.44      0.54      0.48        13

    micro avg       0.42      0.42      0.42        26
    macro avg       0.42      0.42      0.42        26
 weighted avg       0.42      0.42      0.42        26

(444,)
[ 1.15432772e-02 -1.

training: 40 testing: 28
correctsource    29
missed           11
Name: ctl_miss_ws_cs, dtype: int64 correctsource    21
missed            7
Name: ctl_miss_ws_cs, dtype: int64
[0.57142857 0.83333333 0.66666667 0.66666667 0.4        0.8
 0.6       ]
Accuracy: 0.65
               precision    recall  f1-score   support

correctsource       0.73      0.83      0.77        29
       missed       0.29      0.18      0.22        11

    micro avg       0.65      0.65      0.65        40
    macro avg       0.51      0.50      0.50        40
 weighted avg       0.61      0.65      0.62        40

accuracy = 0.75
               precision    recall  f1-score   support

correctsource       0.79      0.90      0.84        21
       missed       0.50      0.29      0.36         7

    micro avg       0.75      0.75      0.75        28
    macro avg       0.65      0.60      0.60        28
 weighted avg       0.72      0.75      0.72        28

(444,)
[-2.96260662e-02  2.59413197e-02  1.14948829e-02

training: 37 testing: 26
correctsource    21
missed           16
Name: ctl_miss_ws_cs, dtype: int64 correctsource    14
missed           12
Name: ctl_miss_ws_cs, dtype: int64
[0.5        0.33333333 0.6        0.6        0.6        0.6
 0.6       ]
Accuracy: 0.5405405405405406
               precision    recall  f1-score   support

correctsource       0.58      0.67      0.62        21
       missed       0.46      0.38      0.41        16

    micro avg       0.54      0.54      0.54        37
    macro avg       0.52      0.52      0.52        37
 weighted avg       0.53      0.54      0.53        37

accuracy = 0.5769230769230769
               precision    recall  f1-score   support

correctsource       0.59      0.71      0.65        14
       missed       0.56      0.42      0.48        12

    micro avg       0.58      0.58      0.58        26
    macro avg       0.57      0.57      0.56        26
 weighted avg       0.57      0.58      0.57        26

(444,)
[ 3.06314034e-02  2.